## Combinatorics of Nonograms

In [1]:
### Imports and math functions
from scipy.special import betainc, gamma, gammainc, hyp2f1
from math import comb as binom
import random

def fibonacci(n: int) -> int:
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

def floor(n: float) -> int:
    return int(n+0.5)


### Functions to calculate the number of distinct possible hints for a line of length n
def k_max(n) -> int:
    return floor((n+1)/2)

def hints_blocks(n: int,k: int):
    return binom(n-(k-1),k)

def hints(n: int):
    return fibonacci(n+2)


### Functions that create a list of every possible hint for line lengths n between some n_min and n_max
def hints_get_next(n: int, prev_hints: list[list[int]]) -> list[list[int]]:
    result = []
    for hint in prev_hints:
        result.append(hint)
        total = sum(hint) + len(hint) 
        if total == n:
            result.append(hint[0:-1] + [hint[-1]+1])
        elif total == n - 1:
            result.append(hint + [1])
    
    return result
    
def hints_list_all(n_min: int, n_max: int, empty_hint_as_zero = False) -> list[list[list[int]]]:
    if empty_hint_as_zero:
        result: list[list[list[int]]] = [[[0]]] #n=0
        result.append([[0], [1]])               #n=1
        result.append([[0], [1], [2]])          #n=2
    else:
        result: list[list[list[int]]] = [[[]]] #n=0
        result.append([[], [1]])               #n=1
        result.append([[], [1], [2]])          #n=2

    if n_max <= 2:
        return result[n_min:n_max+1]
    
    for n in range(3, n_max+1):
        result.append(hints_get_next(n, result[n-1]))

    return result[n_min::]


### Function to calculate the number of lines that match a given hint (needs empty hint as zero!)
def num_lines_matching_hint(hint: list[int], n: int) -> int:
    if len(hint) == 0:
        return 1
    res = binom(n-sum(hint)+1,len(hint))
    return res


### Function to calculate the maximum number of lines that a single hint for a line of length n can describe
def max_num_lines_matching_line_hint(n: int) -> int:
    # https://oeis.org/A073028
    cmax_fibn = [1, 1, 2, 3, 4, 6, 10, 15, 21, 35, 56, 84, 126, 210, 330, 495, 792, 1287, 2002, 3003, 5005, 8008, 12376, 19448, 31824, 50388, 77520, 125970, 203490, 319770, 497420, 817190, 1307504, 2042975, 3268760, 5311735, 8436285, 13123110, 21474180, 34597290]
    assert(n < len(cmax_fibn))
    return cmax_fibn[n]


### Function to calculate the number of nonograms with given dimensions n and m
def num_nonograms(n: int, m: int) -> int:
    return hints(n)**m*hints(m)**n


### Function to generate a random, possible square nonogram from given hints list
def generate_possible_nonogram(n: int, hints_list: list, num_hints: int) -> tuple:
    row_hints = []
    row_hint_sum = 0
    # Randomly sample row hints ensuring the sum is not zero
    while row_hint_sum == 0:
        row_hints = [random.randint(0, num_hints - 1) for _ in range(n)]
        row_hint_sum = sum(sum(hints_list[row_hints[row]]) for row in range(n))

    col_hints = []
    col_hint_sum = 0
    # Randomly sample col hints ensuring the sum matches 
    while col_hint_sum != row_hint_sum:
        col_hints = [random.randint(0, num_hints - 1) for _ in range(n)]
        col_hint_sum = sum(sum(hints_list[col_hints[col]]) for col in range(n))

    return row_hints, col_hints



## Sanity Checks

In [ ]:
n_max = 27
for n in range(n_max + 1):
    k_max_n = k_max(n)
    sumk = 0
    for k in range(k_max_n + 1):
        sumk += hints_blocks(n, k)
    assert(sumk == hints(n))

In [ ]:

n_max = 27
hints_list = hints_list_all(0,n_max)
for n in range(0, n_max+1):
    assert(len(hints_list[n]) == hints(n))
    # print(f"n={n} ({len(hints_list[n])}, {hints(n)}): {hints_list[n]}")


In [ ]:

n_max = 27
hints_list = hints_list_all(0,n_max)
for n in range(0, n_max+1):
    total = sum([num_lines_matching_hint(hint, n) for hint in hints_list[n]])
    # print(log2(total))
    assert(total== 2**n)


In [ ]:

n_max = 40
hints_list = hints_list_all(0,n_max)
for n in range(0, n_max+1):
    highest = max([num_lines_matching_hint(hint, n) for hint in hints_list[n]])
    # print(highest)
    assert(highest == max_num_lines_matching_line_hint(n))


In [ ]:

n_max = 13
for n in range(0, n_max):
    print("n=", n, ":", num_nonograms(n, n), "nonograms,", 2**(n*n), "grids")


## Plot of all hints and their multiplicity

In [8]:
### Create a plot of all hints up until some maximum n
n_max = 8
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np

fig, ax = plt.subplots()
fig.set_size_inches(11,10)
fig.set_dpi(120)

ns = []
hs_fit = []
hs_nofit = []
bottom = np.zeros(n_max)
for i in range(1, n_max + 1):
    ns.append(i)

hints_list = hints_list_all(1,n_max+1)

fitting_hints = []
non_fitting_hints = []

for n in ns:
    fitting_hints.append([])
    non_fitting_hints.append([])
    for h in hints_list[n-1]:
        num = num_lines_matching_hint(h, n)
        if n == sum(h)+len(h)-1 or len(h) == 0:
            fitting_hints[-1].append((h,num))
            # print(f"hint {h} fits in {n}")
        else:
            non_fitting_hints[-1].append((h,num))
            # print(f"hint {h} does not fit in {n}")
    hs_fit.append(len(fitting_hints[-1]))
    hs_nofit.append(len(non_fitting_hints[-1]))

non_fitting_hints_sorted = [[]] + [sorted(sublist, key=lambda x: x[1]) for sublist in non_fitting_hints[1:]]
# debug code to show most ambiguous hint for each n
# for n in ns:
#     print(non_fitting_hints_sorted[n][-1])
# return
plt.bar(ns, hs_fit, label="unambiguous hints", width=0.975, color="#5866DF", zorder=3)
plt.bar(ns, hs_nofit, label="ambiguous hints", width=0.975, bottom=hs_fit, color="#C9791C", zorder=3)
for n in ns:
    assert(hints(n) == len(fitting_hints[n-1]) + len(non_fitting_hints[n-1]))
    if fitting_hints[n-1]:
        for y, h in enumerate(fitting_hints[n-1]):
            hinttext = str('.'.join(map(str, h[0]))) if h[0] else "0"
            if h[0] and sum(h[0]) + max(h[0]) + len(h[0]) - 1 <= n:
                hinttext += "${}_f$"
            plt.text(n-0.35, y+0.2, hinttext)
            plt.text(n+0.45, y+0.2, str(h[1]), size='smaller', color="#2F2F2F", horizontalalignment='right')
    for y, h in enumerate(non_fitting_hints_sorted[n-1]):
        hinttext = str('.'.join(map(str, h[0]))) if h[0] else "0"
        if h[0] and sum(h[0]) + max(h[0]) + len(h[0]) - 1 <= n:
            hinttext += "${}_f$"
        plt.text(n-0.35, y+0.2+hs_fit[n-1], hinttext)
        plt.text(n+0.45, y+0.2+hs_fit[n-1], str(h[1]), size='smaller', color="#2F2F2F", horizontalalignment='right')


# Set major ticks and grid
ax.yaxis.set_major_locator(MultipleLocator(10))
ax.grid(which='major', linestyle='-', linewidth='0.5', color='grey')

# Set minor ticks and grid
ax.yaxis.set_minor_locator(MultipleLocator(5))
ax.grid(which='minor', linestyle='--', linewidth='0.5', color='grey')

_ = plt.xlabel("Line Length $n$")
_ = plt.ylabel("Number of possible Nonogram hints")

plt.legend()
plt.annotate(r"$F_{n+2}$", xy=(0.8, 0.75), xycoords='figure fraction', rotation=270.0,size='larger')
plt.annotate(r"$F_{n} + 1$", xy=(0.805, 0.30), xycoords='figure fraction', rotation=270.0, size='larger')
x1, y1 = [8.55, 8.55], [0, 55]
x2, y2 = [8.65, 8.65], [0, 22]
plt.plot(x1, y1, x2, y2, marker = '_', color='black')
# x3, y3 = [4.1, 4.4], [17.5,8.7]
# x4, y4 = [4.8, 5.4], [17,13.5]
# x5, y5 = [3.5, 3.4], [17,6.1]
# plt.plot(x3, y3, x4, y4,x5, y5, marker = '', color="#2F2F2F", linewidth=1)
# plt.annotate("Number of lines matching the hint", xy=(2.89,18),  color="#2F2F2F")
_=plt.xlim(0.5, 8.9)

plt.savefig("hints_list_small_n.pdf", bbox_inches='tight')
plt.close()

## Sampling of all possible Nonograms

In [ ]:
### Sample all square nonograms up until some n_max and test if they are possible and/or solvable
n_max = 4
from itertools import product
from clingo import Control

hints_list = hints_list_all(0,n_max, empty_hint_as_zero=True)
for n in range(1,n_max+1):
    count_total = 0
    count_possible = 0
    count_not_possible = 0
    count_no_soln = 0
    count_soln = 0
    count_soln_unique = 0
    count_soln_multiple = 0
    
    num_hints = hints(n)
    hint_combinations = product(range(num_hints), repeat=n) # cartesian products {0..h-1}^n
    hint_combinations2 = product(range(num_hints), repeat=n)
    for row_hints in hint_combinations:
        row_hint_sum = 0
        for row in range(0,n):
            row_hint_sum += sum(hints_list[n][row_hints[row]])

        # for row in range(0,n):
        #     print(f"row {row+1}:", hints_list[n][row_hints[row]])
        # print(f"row sum =", row_hint_sum)

        hint_combinations2 = product(range(num_hints), repeat=n)
        for col_hints in hint_combinations2:
            col_hint_sum = 0
            for col in range(0,n):
                col_hint_sum += sum(hints_list[n][col_hints[col]])

            # for col in range(0,n):
            #     print(f"\tcol {col+1}:", hints_list[n][col_hints[col]])
            # print(f"col sum =", col_hint_sum)
            count_total += 1
            if col_hint_sum == row_hint_sum:
                count_possible += 1

                # Initialize the clingo control specifying two models are required, and give the dimensional constants
                ctl = Control(["2",
                            "-c", f"w={n}", 
                            "-c", f"h={n}"])

                # Add the hint predicates to the base program
                for row_index in range(n):
                    for hint_index, hint_length in enumerate(hints_list[n][row_hints[row_index]]):
                        ctl.add(f"row_hint({row_index+1},{hint_index+1},{hint_length}).")
                for col_index in range(n):
                    for hint_index, hint_length in enumerate(hints_list[n][col_hints[col_index]]):
                        ctl.add(f"col_hint({col_index+1},{hint_index+1},{hint_length}).")
                
                # Load the solver file
                ctl.load("../solvers/sbs-improved.lp")

                # Find the first model and track the computation times
                ctl.ground([("base", [])])
                handler = ctl.solve(yield_=True)
                model = handler.model()
                handler.resume()
                model2 = handler.model()
                # Console output
                if not model:
                    count_no_soln += 1
                else:
                    count_soln += 1
                    if not model2:
                        count_soln_unique += 1
                    else:
                        count_soln_multiple += 1

            else:
                count_not_possible += 1
        #     print()
        # print()

    print(f"n={n} hints are {hints_list[n]}")
    print(f"\t#total nonograms = {count_total} ({(num_hints**n)**2})")
    print(f"\t#possible nonograms = {count_possible} ({count_possible/count_total})")
    print(f"\t#impossible nonograms = {count_not_possible} ({count_not_possible/count_total})")
    print(f"\t#nonograms with no solution = {count_no_soln} ({count_no_soln/count_possible})")
    print(f"\t#nonograms with a solution = {count_soln} ({count_soln/count_possible})")
    print(f"\t#nonograms with a unique solution = {count_soln_unique} ({count_soln_unique/count_soln})")
    print(f"\t#nonograms with multiple solutions = {count_soln_multiple} ({count_soln_multiple/count_soln})")


In [ ]:
### Function that samples N square nonograms of size n and counts the number of possible, solvable, unique and multiple solutions
from clingo import Control

def sample_nonograms(n: int, hints_list: list, num_hints: int, N: int) -> dict:
    count_total = 0
    count_possible = 0
    count_no_soln = 0
    count_soln = 0
    count_soln_unique = 0
    count_soln_multiple = 0

    for _ in range(N):
        row_hints, col_hints = generate_possible_nonogram(n, hints_list, num_hints)

        count_total += 1
        count_possible += 1

        # Initialize the clingo control specifying two models are required, and give the dimensional constants
        ctl = Control(["2",
                       "-c", f"w={n}",
                       "-c", f"h={n}"])

        # Add the hint predicates to the base program
        for row_index in range(n):
            for hint_index, hint_length in enumerate(hints_list[row_hints[row_index]]):
                ctl.add(f"row_hint({row_index+1},{hint_index+1},{hint_length}).")
        for col_index in range(n):
            for hint_index, hint_length in enumerate(hints_list[col_hints[col_index]]):
                ctl.add(f"col_hint({col_index+1},{hint_index+1},{hint_length}).")

        # Load the solver file
        ctl.load("../solvers/sbs-improved.lp")

        # Find the first model and track the computation times
        ctl.ground([("base", [])])
        handler = ctl.solve(yield_=True)
        model = handler.model()
        handler.resume()
        model2 = handler.model()
        # Console output
        if not model:
            count_no_soln += 1
        else:
            count_soln += 1
            if not model2:
                count_soln_unique += 1
            else:
                count_soln_multiple += 1

    return {
        'count_total': count_total,
        'count_possible': count_possible,
        'count_no_soln': count_no_soln,
        'count_soln': count_soln,
        'count_soln_unique': count_soln_unique,
        'count_soln_multiple': count_soln_multiple
    }


In [ ]:
### Sample random square nonograms up until some n_max and test if they are possible and/or solvable, using all logical cores
### !Very dumb and inefficient, finds almost no solvable nonograms because they are so exceedingly rare for n>5!
import concurrent.futures

n_max = 6
N = 1000000  # Number of nonograms to sample

hints_list = hints_list_all(0, n_max, empty_hint_as_zero=True)
# Determine the number of cores
num_cores = concurrent.futures.ThreadPoolExecutor()._max_workers

for n in range(n_max, n_max + 1):

    num_hints = hints(n)
    assert(len(hints_list[n]) == num_hints)

    # Create a ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_cores) as executor:
        # Submit tasks to the executor
        futures = [executor.submit(sample_nonograms, n, hints_list[n], num_hints, N // num_cores) for _ in range(num_cores)]

        # Aggregate the results
        count_total = 0
        count_possible = 0
        count_no_soln = 0
        count_soln = 0
        count_soln_unique = 0
        count_soln_multiple = 0

        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            count_total += result['count_total']
            count_possible += result['count_possible']
            count_no_soln += result['count_no_soln']
            count_soln += result['count_soln']
            count_soln_unique += result['count_soln_unique']
            count_soln_multiple += result['count_soln_multiple']

    print(f"n={n} hints are {hints_list[n][:10]}... (total {num_hints})")
    print(f"\t#total nonograms sampled = {count_total} ({(num_hints**n)**2})")
    print(f"\t#possible nonograms = {count_possible} ({count_possible/count_total})")
    print(f"\t#nonograms with no solution = {count_no_soln} ({count_no_soln/count_possible if count_possible > 0 else 0})")
    print(f"\t#nonograms with a solution = {count_soln} ({count_soln/count_possible if count_possible > 0 else 0})")
    print(f"\t#nonograms with a unique solution = {count_soln_unique} ({count_soln_unique/count_soln if count_soln > 0 else 0})")
    print(f"\t#nonograms with multiple solutions = {count_soln_multiple} ({count_soln_multiple/count_soln if count_soln > 0 else 0})")


In [ ]:
### Sample a single possible nonogram of size n and print the row and column hints, and check if it is solvable
from clingo import Control

n = 8
hints_list = hints_list_all(0, n, empty_hint_as_zero=True)
num_hints = hints(n)

row_hints, col_hints = generate_possible_nonogram(n, hints_list[n], num_hints)
print(f"Row hints: {[hints_list[n][row_hints[i]] for i in range(n)]}, Column hints: {[hints_list[n][col_hints[j]] for j in range(n)]}")
print(f"Row hint sum: {sum(sum(hints_list[n][row_hints[i]]) for i in range(n))}, Column hint sum: {sum(sum(hints_list[n][col_hints[j]]) for j in range(n))} ")

# Initialize the clingo control specifying two models are required, and give the dimensional constants
ctl = Control(["2",
                "-c", f"w={n}",
                "-c", f"h={n}"])

# Add the hint predicates to the base program
for row_index in range(n):
    for hint_index, hint_length in enumerate(hints_list[n][row_hints[row_index]]):
        ctl.add(f"row_hint({row_index+1},{hint_index+1},{hint_length}).")
for col_index in range(n):
    for hint_index, hint_length in enumerate(hints_list[n][col_hints[col_index]]):
        ctl.add(f"col_hint({col_index+1},{hint_index+1},{hint_length}).")

# Load the solver file
ctl.load("../solvers/sbs-improved.lp")

# Find the first model and track the computation times
ctl.ground([("base", [])])
handler = ctl.solve(yield_=True)
model = handler.model()
handler.resume()
model2 = handler.model()
# Console output
if model:
    print("Found a solution")
    if model2:
        print("Found a second solution")
    else:  
        print("Found a unique solution")
else:
    print("No solution found")


## Sample all grids & convert to nonograms